
# VN Jobs 2024 — End‑to‑End: Làm sạch • Trực quan • Prophet • ML

**Mục tiêu:** Gom toàn bộ pipeline vào một notebook: làm sạch & chuẩn hoá, EDA, Prophet (nếu có cột ngày), và mô hình dự đoán lương.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

pd.set_option("display.max_colwidth", 300)
DATA_PATH = Path("C:/Users/Lapto/JobsTrending/datasets/short_jobs/train.csv")
print("Using data file:", DATA_PATH.resolve())


SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (990851056.py, line 9)

In [ ]:

# Đọc dữ liệu (thử nhiều encoding)
encodings_to_try = ["utf-8", "utf-8-sig", "cp1258", "cp1252"]
df = None
last_err = None
for enc in encodings_to_try:
    try:
        df = pd.read_csv(DATA_PATH, encoding=enc, low_memory=False)
        print("Read with encoding:", enc, "| shape:", df.shape)
        break
    except Exception as e:
        last_err = e

if df is None:
    raise RuntimeError(f"Không thể đọc CSV. Lỗi cuối: {last_err}")

print("Columns:", list(df.columns))
df.head(5)

In [ ]:

# Helpers + hằng số
def normalize_text(s: pd.Series) -> pd.Series:
    return s.astype(str).str.strip().str.lower().str.replace(r"\s+", " ", regex=True)

def parse_years(exp: str) -> float:
    import re
    if not isinstance(exp, str) or exp.strip()=="" or "không yêu cầu" in exp:
        return 0.0
    nums = re.findall(r"\d+\.?\d*", exp)
    if not nums:
        return np.nan
    return float(nums[0])

USD_TO_MILLION_VND = 0.025  # 1 USD ~ 0.025 triệu VND


In [ ]:

# Làm sạch & chuẩn hoá
for c in ["job_title","job_type","position_level","city","experience","skills","job_fields","unit"]:
    if c in df.columns:
        df[c] = normalize_text(df[c])

for c in ["salary_min","salary_max"]:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce")

if "unit" in df.columns:
    usd_mask = df["unit"].fillna("").str.contains("usd", case=False)
else:
    usd_mask = pd.Series(False, index=df.index)

df.loc[usd_mask, "salary_min"] = df.loc[usd_mask, "salary_min"] * USD_TO_MILLION_VND
df.loc[usd_mask, "salary_max"] = df.loc[usd_mask, "salary_max"] * USD_TO_MILLION_VND

df["salary_min"] = df["salary_min"].clip(lower=0.1)
df["salary_max"] = df[["salary_max","salary_min"]].max(axis=1).clip(lower=0.1)
df["salary_mid"] = df[["salary_min","salary_max"]].mean(axis=1)

if "experience" in df.columns:
    df["experience_years"] = df["experience"].apply(parse_years)
else:
    df["experience_years"] = np.nan

if "skills" in df.columns:
    df["skills_count"] = df["skills"].fillna("").apply(lambda s: len([x.strip() for x in str(s).split(",") if x.strip()!='']))
else:
    df["skills_count"] = np.nan

df.head(3)

In [ ]:

# Lưu dữ liệu sạch
out_clean = Path("clean_jobs.csv")
df.to_csv(out_clean, index=False, encoding="utf-8")
print("Đã lưu:", out_clean.resolve())
df[["salary_min","salary_max","salary_mid"]].describe()

## Trực quan — Top thành phố theo số tin đăng

In [ ]:

if "city" in df.columns:
    top_city = df["city"].value_counts().head(20)
    plt.figure(figsize=(10,5))
    top_city.plot(kind="bar")
    plt.title("Top 20 thành phố theo số tin đăng")
    plt.xlabel("Thành phố")
    plt.ylabel("Số tin")
    plt.tight_layout()
    plt.show()
else:
    print("Không có cột 'city'.")

## Trực quan — Phân phối lương (triệu VND/tháng)

In [ ]:

if "salary_mid" in df.columns:
    vals = df["salary_mid"].dropna().values
    if len(vals) > 0:
        plt.figure(figsize=(8,4))
        plt.hist(vals, bins=40)
        plt.title("Phân phối lương (triệu VND/tháng)")
        plt.xlabel("Lương (triệu VND/tháng)")
        plt.ylabel("Số lượng tin")
        plt.tight_layout()
        plt.show()
    else:
        print("Không có salary_mid hợp lệ.")
else:
    print("Không có cột 'salary_mid'.")

## (Tuỳ chọn) Prophet — Dự báo số tin đăng 90 ngày

In [ ]:

# Dò cột ngày
date_col = None
for c in df.columns:
    lc = c.lower()
    if any(k in lc for k in ["date","posted","time","created"]):
        date_col = c
        break

if date_col is None:
    print("Chưa phát hiện cột thời gian. Bỏ qua Prophet.")
else:
    print("Sử dụng cột ngày:", date_col)
    try:
        from prophet import Prophet
        ts = df[[date_col]].copy()
        ts[date_col] = pd.to_datetime(ts[date_col], errors="coerce")
        ts = ts.dropna()
        if len(ts)==0:
            print("Không có ngày hợp lệ sau chuyển đổi.")
        else:
            ts["ds"] = ts[date_col]
            ts["y"] = 1
            ts = ts.groupby("ds").agg(y=("y","sum")).reset_index()
            m = Prophet()
            m.fit(ts)
            future = m.make_future_dataframe(periods=90)
            fcst = m.predict(future)
            plt.figure(figsize=(10,4))
            plt.plot(fcst["ds"], fcst["yhat"])
            plt.title("Dự báo số tin đăng (Prophet) — yhat")
            plt.xlabel("Ngày")
            plt.ylabel("Số tin (ước lượng)")
            plt.tight_layout()
            plt.show()
            fcst.tail(10)
    except Exception as e:
        print("Không chạy được Prophet. Hãy cài 'prophet'. Lý do:", e)

## Mô hình dự đoán lương — RandomForest

In [ ]:

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score
import joblib, json

y = pd.to_numeric(df["salary_mid"], errors="coerce")
text_cols = [c for c in ["job_title","skills","job_fields","info"] if c in df.columns]
cat_cols = [c for c in ["city","position_level","job_type"] if c in df.columns]
num_cols = [c for c in ["experience_years","skills_count","salary_min","salary_max"] if c in df.columns]

valid = y.notna()
X = df.loc[valid, text_cols + cat_cols + num_cols].copy()
y = y.loc[valid]

if text_cols:
    X["text_merged"] = X[text_cols].fillna("").agg(" ".join, axis=1)
for c in cat_cols:
    X[c] = X[c].fillna("na")
for c in num_cols:
    X[c] = pd.to_numeric(X[c], errors="coerce").fillna(0.0)

preprocess = ColumnTransformer(
    transformers=[
        ("text", TfidfVectorizer(max_features=1000, ngram_range=(1,2)), "text_merged")
    ] + ([("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols)] if cat_cols else []) +     ([("num","passthrough", num_cols)] if num_cols else []),
    remainder="drop"
)

model = Pipeline([
    ("prep", preprocess),
    ("rf", RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1))
])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model.fit(X_train, y_train)
pred = model.predict(X_test)
mae = float(mean_absolute_error(y_test, pred))
r2 = float(r2_score(y_test, pred))
print("MAE (triệu VND):", round(mae,2))
print("R²:", round(r2,3))

joblib.dump(model, "salary_model.joblib")
with open("feature_spec.json", "w", encoding="utf-8") as f:
    json.dump({
        "text_cols": text_cols,
        "cat_cols": cat_cols,
        "num_cols": num_cols,
        "target": "salary_mid",
        "unit": "triệu VND/tháng",
        "MAE": mae,
        "R2": r2
    }, f, ensure_ascii=False, indent=2)

print("Đã lưu: salary_model.joblib, feature_spec.json")